<a href="https://colab.research.google.com/github/ngockhanh5110/nlp-vietnamese-text-summarization/blob/main/notebooks/inferencing_vietnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Authenticate
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
OUTPUT_DIR = '/content/training/'

In [4]:
import random

random.seed(0)

In [5]:
%%capture

!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

In [6]:
%%capture

# !wget 'https://github.com/CLC-HCMUS/ViMs-Dataset/raw/master/ViMs.zip'
# !unzip 'ViMs.zip'

# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

In [7]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *

## Processing data

In [8]:
# from vncorenlp import VnCoreNLP
# rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [9]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('/content/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('/content/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('/content/vietnews-master/data/test_tokenized/*')

In [10]:
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ' '.join(''.join(rows[4:]).split('\n'))
    summary = ' '.join(rows[2].split('\n'))
          
  return {'file' : pathfile,
          'original': original, 
          'summary': summary}

In [11]:
read_content(train_paths[0])

{'file': '/content/vietnews-master/data/train_tokenized/033889.txt.seg',
 'original': 'Theo RT , liên_quân do Mỹ dẫn_đầu được cho là đã không_kích các vị_trí của quân_đội Syria tại khu_vực Al_Bukamal thuộc tỉnh Deir ez - Zor , phía Đông_Syria . Thông_tin trên được hãng thông_tấn nhà_nước Syria SANA trích_dẫn nguồn tin quân_sự cho_hay . Theo đó , cuộc không_kích nhằm vào một_số vị_trí của quân_đội Syria tại al - Harra , phía Đông nam khu_vực Al_Bukamal . Theo nguồn_tin này , vụ không_kích dẫn đến tổn_thất về nhân_lực của quân_đội Syria , tuy_nhiên con_số thương_vong cụ_thể chưa được công_bố . Vụ không_kích này có_thể do một_số máy_bay_không_người_lái ( UAV ) của Mỹ thực_hiện , đồng_thời vụ không_kích này được cho là nhằm vào một_số vị_trí của lực_lượng Iraq và quân_đội Chính_phủ Syria tại khu_vực giữa Al_Bukamal và Tanf . Tuy_nhiên , hiện Lầu_Năm_Góc phủ_nhận có liên_quan tới vụ_việc . Đây không phải là vụ không kích đầu_tiên nhằm vào lực_lượng quân_đội Syria và các lực_lượng ủng_hộ Chí

In [12]:
def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)
  
  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace = True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df

In [13]:
# train_df = get_dataframe(train_paths)

In [14]:
test_df = get_dataframe(test_paths)

## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [15]:
%%capture
!pip install datasets==1.0.2
!pip install transformers

import datasets
import transformers

In [16]:
from transformers import RobertaTokenizerFast,AutoTokenizer


In [17]:
from sklearn.model_selection import train_test_split

# train_data =  Dataset.from_pandas(train_df)

In [18]:
test_data =  Dataset.from_pandas(test_df)

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [19]:
%%capture
!rm seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py

!pip install git-python==1.0.3
!pip install sacrebleu==1.4.12
!pip install rouge_score

from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [20]:
import datasets

In [21]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [24]:
!mkdir 'training'

In [215]:
!gsutil -m cp -r 'gs://kaggle-vbdi-test/training_data/checkpoint-20000/*' '/content/training/' 

Copying gs://kaggle-vbdi-test/training_data/checkpoint-20000/config.json...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-20000/optimizer.pt...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-20000/pytorch_model.bin...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-20000/scheduler.pt...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-20000/trainer_state.json...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-20000/training_args.bin...
\ [6/6 files][  1.8 GiB/  1.8 GiB] 100% Done  22.8 MiB/s ETA 00:00:00           
Operation completed over 6 objects/1.8 GiB.                                      


In [216]:
import datasets
from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

model = EncoderDecoderModel.from_pretrained('/content/training')
model.to("cuda")

# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 32  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

In [217]:
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1","rouge2","rougeL"])

In [218]:
for key,value in rouge_output.items():
  print(key)
  print(value.mid)

rouge1
Score(precision=0.633608207130866, recall=0.5650859859756249, fmeasure=0.5866591733414444)
rouge2
Score(precision=0.29153183231813984, recall=0.2602756399045278, fmeasure=0.2699654296173031)
rougeL
Score(precision=0.40849614549773827, recall=0.3648733690442705, fmeasure=0.37846726543911663)


In [231]:
i = 39
print('Prediction: ',pred_str[i])
print('Truth: ',label_str[i])
print('Content: ',test_data[i]['original'])

Prediction:  Tổng_thống Indonesia Joko_Widodo cho_biết nước này sẵn_sàng tuần_tra chung với Australia trên Biển_Đông, nhưng không nêu vấn_đề này.
Truth:  Tổng_thống Indonesia cho_biết ông sẽ thảo_luận việc tuần_tra chung ở Biển_Đông khi gặp Thủ_tướng Australia cuối tuần này . 
Content:  Tổng_thống Indonesia_Widodo đứng trên boong chiến_hạm Hải_quân Indonesia . " Nếu không có căng_thẳng , tôi nghĩ việc tuần_tra chung rất quan_trọng . Chúng_tôi sẽ thảo_luận vấn_đề này với Thủ_tướng Turnbull " , Reuters hôm_nay dẫn lời Tổng_thống Indonesia Joko_Widodo . Tổng_thống Indonesia_Widodo sẽ có cuộc gặp với Thủ_tướng Australia_Malcolm_Turnbull vào cuối tuần này . Widodo nói ông muốn tuần_tra chung với Australia trên Biển_Đông , nhưng chỉ khi điều này không làm bùng_phát căng_thẳng với Trung_Quốc . Indonesia có truyền_thống lựa_chọn lập_trường trung_lập trong vấn_đề Biển_Đông . Tuy_nhiên , sau khi Trung_Quốc chọc_giận Indonesia bằng tuyên_bố hai nước có " tuyên_bố chồng_lấn " về vùng_biển gần đảo 

In [34]:
test_data[i]['file']

'/content/vietnews-master/data/test_tokenized/016385.txt.seg'

In [36]:
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [212]:
text = '''
Sáng 3/2, Chủ tịch UBND thành phố Hà Nội Chu Ngọc Anh, Trưởng đoàn kiểm tra số 2 của Ban Thường vụ Thành ủy đã đến kiểm tra công tác phòng chống dịch COVID-19 tại huyện Thanh Oai và Chương Mỹ.

 Tại 2 huyện, Chủ tịch UBND thành phố Hà Nội Chu Ngọc Anh cho biết, thời gian qua thành phố đã bám sát chỉ đạo của T.Ư và đã kịp thời lãnh đạo, chỉ đạo, kích hoạt hệ thống phòng chống dịch từ thành phố đến khu dân cư, tổ dân phố với tinh thần: “Trách nhiệm cao nhất, chủ động quyết liệt, không lơi là chủ quan để kiểm soát, không chế dịch bệnh nhanh nhất”
'''

In [213]:
text = rdrsegmenter.tokenize(text)
text = ' '.join([' '.join(x) for x in text])
text

'Sáng 3/2 , Chủ_tịch UBND thành_phố Hà_Nội Chu_Ngọc_Anh , Trưởng_đoàn kiểm_tra số 2 của Ban Thường_vụ Thành_uỷ đã đến kiểm_tra công_tác phòng_chống dịch COVID-19 tại huyện Thanh_Oai và Chương_Mỹ . Tại 2 huyện , Chủ_tịch UBND thành_phố Hà_Nội Chu_Ngọc_Anh cho biết , thời_gian qua thành_phố đã bám sát chỉ_đạo của T.Ư và đã kịp_thời lãnh_đạo , chỉ_đạo , kích_hoạt hệ_thống phòng_chống dịch từ thành_phố đến khu dân_cư , tổ_dân_phố với tinh_thần : “ Trách_nhiệm cao nhất , chủ_động quyết_liệt , không lơi_là chủ_quan để kiểm_soát , không chế dịch_bệnh nhanh nhất ”'

In [214]:
inputs = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
input_ids = inputs.input_ids.to("cuda")
attention_mask = inputs.attention_mask.to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask)

# all special tokens including will be removed
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str

['Chủ_tịch là Chủ_tịch UBND TP Hà_Nội Chu_Ngọc_Anh khẳng_định, hệ_thống phòng_chống dịch_bệnh COVINA đã được vận_hành thông_suốt, không chủ_quan chủ_quan, chủ_quan chủ_quan mà chủ_quan chủ_quan chủ_quan.']